In [ ]:
import os
import pandas as pd
import glob
import numpy as np
import pandas as pd
import csv
from math import sqrt

# Change Data Access Location For Google Drive


In [ ]:
os.chdir('/content/drive/My Drive/New LSTM Crack')

# Load in Data


In [ ]:
# Temperature Ansan Generated Data at
measuredtemp = 150
# Convert to Kelvin
temp = measuredtemp + 273.15
# Force Loading the Data was Run at
Force = 1600
#folder = "./NewData/HeatTreated/160C/1800N/t150_4-12-21_R05_1800N"
folder = "./NewData/NonHeatTreated/150C/2-16-1600N-150T"

In [ ]:
#Get all files
working_files = glob.glob(folder + '/*')
#Create Dataframe to put data in
all_dat = pd.DataFrame()

Read All Data into DataFrame

In [ ]:
for file in working_files:
  df = pd.read_csv(file, sep="\t")
  all_dat = all_dat.append(df, ignore_index=True)

Sort data by time since it may have been read in out of order

In [ ]:
all_dat = all_dat.sort_values(by=['# time(sec)'])

        # time(sec)  crack length FEA(mm)
109518    175.97656              0.192386
109519    176.17188              0.181693
109520    176.36719              0.191780
109521    176.56250              0.184170
109522    176.75781              0.130154
...             ...                   ...
5626    53645.11750              3.023230
5627    53645.31250              3.072577
5628    53645.50750              3.024461
5629    53645.70350              3.018152
5630    53645.89850              3.033904

[196519 rows x 2 columns]


Rename column headers

In [ ]:
all_dat = all_dat.rename(columns={'# time(sec)': 'Time', 'crack length FEA(mm)': 'Crack Length'})

               Time  Crack Length
109518    175.97656      0.192386
109519    176.17188      0.181693
109520    176.36719      0.191780
109521    176.56250      0.184170
109522    176.75781      0.130154
...             ...           ...
5626    53645.11750      3.023230
5627    53645.31250      3.072577
5628    53645.50750      3.024461
5629    53645.70350      3.018152
5630    53645.89850      3.033904

[196519 rows x 2 columns]


Generate the Force and Temperature Columns and add them in

In [ ]:
force_array = np.full((len(all_dat),1), Force)
temp_array = np.full((len(all_dat), 1), temp)
all_dat.insert(2, "Force",force_array,True)
all_dat.insert(3, "Temp",temp_array,True)

# Generate Testing and Training Data

Can be used to Limit how much data is actually utilized for training file

In [ ]:
Training = all_dat[:]

Formatting File Names

In [ ]:
train_filename = folder + '_training.csv'
test_filename = folder + '_testing.csv'
predict_name = folder.split('/')[-1]
train_filename = train_filename.replace('NewData','Training')
test_filename = test_filename.replace('NewData','Test')

train_filename = train_filename.split('/')
train_filename.pop(3)
#train_filename.pop(3)
train_filename='/'.join(train_filename)

test_filename = test_filename.split('/')
test_filename.pop(3)
#test_filename.pop(3)
test_filename='/'.join(test_filename)


# Generate Data used for Prediction

Isolate Force and Temperature data for later use

In [ ]:
force_series = Training.iloc[:,2].to_numpy().reshape((len(Training),1))
temp_series = Training.iloc[:,3].to_numpy().reshape((len(Training),1))

Use force Data To calculate Stress Intensity

In [ ]:
stress_intensity = []
# Get Crack Length
crack_series = Training.iloc[:,1].to_numpy().reshape((len(Training),1))
# Values taken from paper
specimen_width = 0.0408 #m
crossSectArea = 2.6388 #m

# Calculate Stress intensity at every point
# Equation taken from paper
for i in range(len(Training)):
  term1 = (force_series[i] / crossSectArea) * sqrt(3.14 * crack_series[i])
  term2 = 8.574
  term3 = 10.365 * (crack_series[i] / specimen_width)
  term4 = 5.499 * ((crack_series[i] / specimen_width)**2)
  intensity = term1*(term2 - term3 + term4)
  stress_intensity.append(intensity)
  if i % 10000 == 0:
    print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


Compile inputs to save prediction file

In [ ]:
stress_intensity = np.asarray(stress_intensity).reshape(len(Training),1)
series= np.hstack((temp_series, stress_intensity))

# Save All Generated Data

In [ ]:
Training.to_csv(train_filename, index=False)
Training.to_csv(test_filename, index=False)
np.savetxt('./Predict/NonHeatTreated/{}.csv'.format(predict_name), series, delimiter=",", header='Temp, Stress Intensity')